<a href="https://colab.research.google.com/github/Willyzw/monodepth2/blob/master/monodepth2_handson.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Clone repository

In [ ]:
!git clone https://github.com/Willyzw/monodepth2

Cloning into 'monodepth2'...
remote: Enumerating objects: 148, done.
remote: Total 148 (delta 0), reused 0 (delta 0), pack-reused 148
Receiving objects: 100% (148/148), 10.26 MiB | 24.78 MiB/s, done.
Resolving deltas: 100% (71/71), done.


# Network